In [3]:
import numpy as np
import pandas as pd

## 데이터셋 : 타이타닉 데이터셋 (출처:Kaggle.com)

In [4]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [5]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 전처리: train / validation 세트 나누기

1. 먼저 feature와 label을 정의합니다.
2. feature / label을 정햇으면 적절한 비율로 train / validation set을 나눕니다.

In [6]:
feature = ['Pclass', 'Sex', 'Age', 'Fare']
label = ['Survived']

In [7]:
train[feature].head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [8]:
train[label].head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [9]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_vaild, y_train, y_vaild = train_test_split(train[feature], train[label],
                                                   test_size=0.2, shuffle=True,
                                                   random_state=30)

In [14]:
X_train.shape, y_train.shape

((712, 4), (712, 1))

In [15]:
X_vaild.shape, y_vaild.shape

((179, 4), (179, 1))

## 전처리 : 결측치

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [17]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

* 개별 column의 결측치에 대해 확인하는 방법

In [19]:
train['Age'].isnull().sum()

177

### 1. 수치형(Numerical Column) 데이터에 대한 결측치 처리

In [20]:
train['Age'].fillna(0).describe()

count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [21]:
train['Age'].fillna(train['Age'].mean()).describe()

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

imputer : 2개 이상의 column을 한 번에 처리할 때

In [22]:
from sklearn.impute import SimpleImputer

In [23]:
imputer = SimpleImputer(strategy='mean')

fit()을 통해 결측치에 대한 학습을 진행합니다.

In [25]:
imputer.fit(train[['Age','Pclass']])

SimpleImputer()

transform()을 통해 실제 결측치에 대한 처리를 해줍니다.

In [26]:
result = imputer.transform(train[['Age','Pclass']])

In [27]:
result

array([[22.        ,  3.        ],
       [38.        ,  1.        ],
       [26.        ,  3.        ],
       ...,
       [29.69911765,  3.        ],
       [26.        ,  1.        ],
       [32.        ,  3.        ]])

In [28]:
train[['Age', 'Pclass']]= result

In [29]:
train[['Age', 'Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [30]:
train[['Age', 'Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


In [31]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [32]:
train[['Age', 'Pclass']].isnull().sum()

Age       177
Pclass      0
dtype: int64

In [33]:
imputer = SimpleImputer(strategy='median')

In [34]:
result = imputer.fit_transform(train[['Age', 'Pclass']])

In [35]:
train[['Age', 'Pclass']] = result

In [36]:
train[['Age', 'Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

### 2. (Categorical Column)데이터에 대한 결측치 처리

In [37]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

* 1개의 column을 처리하는 경우

In [38]:
train['Embarked'].fillna('S')

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

* Imputer를 사용하여 2개 이상의 columns를 처리

In [39]:
imputer = SimpleImputer(strategy = 'most_frequent')

In [41]:
result = imputer.fit_transform(train[['Embarked', 'Cabin']])

In [43]:
train[['Embarked', 'Cabin']] = result

In [44]:
train[['Embarked', 'Cabin']].isnull().sum()

Embarked    0
Cabin       0
dtype: int64

## Label Encoding: 문자(categorical)를 수치(numerical)로 변환

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [49]:
def convert(data):
    if data == 'male':
        return 1
    elif data == 'female':
        return 0

In [50]:
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [51]:
train['Sex'].apply(convert)

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [52]:
from sklearn.preprocessing import LabelEncoder

In [53]:
le = LabelEncoder()

In [54]:
train['Sex_num'] = le.fit_transform(train['Sex'])

In [55]:
train['Sex_num'].value_counts()

1    577
0    314
Name: Sex_num, dtype: int64

In [56]:
le.classes_

array(['female', 'male'], dtype=object)

In [57]:
le.inverse_transform([0,1,1,0,0,1,1])

array(['female', 'male', 'male', 'female', 'female', 'male', 'male'],
      dtype=object)

NaN값이 포함되어 있다면, LabelEncoder가 정상 작동하지 않는다.

In [58]:
le.fit_transform(train['Embarked'])

array([2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2,
       1, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0,
       1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 0, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 0, 2, 0, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1,
       2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0,
       2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 1,

In [59]:
train['Embarked'] = train['Embarked'].fillna('S')

In [60]:
le.fit_transform(train['Embarked'])

array([2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2,
       1, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0,
       1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 0, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 0, 2, 0, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1,
       2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0,
       2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 1,

## 원 핫 인코딩 (One Hot Encoding)

In [75]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [76]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [77]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [78]:
train['Embarked'] = train['Embarked'].fillna('S')

In [79]:
train['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [80]:
train['Embarked_num'] = LabelEncoder().fit_transform(train['Embarked'])

In [81]:
train['Embarked_num'].value_counts()

2    646
0    168
1     77
Name: Embarked_num, dtype: int64

In [82]:
train['Embarked'][:6]

0    S
1    C
2    S
3    S
4    S
5    Q
Name: Embarked, dtype: object

In [83]:
train['Embarked_num'][:6]

0    2
1    0
2    2
3    2
4    2
5    1
Name: Embarked_num, dtype: int32

In [84]:
pd.get_dummies(train['Embarked_num'][:6])

,0,1,2
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0


In [85]:
one_hot = pd.get_dummies(train['Embarked_num'][:6])

In [86]:
one_hot.columns = ['C', 'Q', 'S']

In [87]:
one_hot

,C,Q,S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0


## 전처리: Normalize(정규화)

* 네이버 영화 평점 (0~10점) : [2,4,6,8,10]
* 넷플릭스 영화 평점 (0~5점) : [1,2,3,4,5]

In [88]:
movie = {'naver': [2,4,6,8,10],
        'netfilx': [1,2,3,4,5]}

In [89]:
movie = pd.DataFrame(data=movie)
movie

,naver,netfilx
0,2,1
1,4,2
2,6,3
3,8,4
4,10,5


In [90]:
from sklearn.preprocessing import MinMaxScaler

In [91]:
min_max_scaler = MinMaxScaler()

In [92]:
min_max_movie = min_max_scaler.fit_transform(movie)

In [93]:
pd.DataFrame(min_max_movie, columns=['naver', 'netflix'])

,naver,netflix
0,0.00,0.00
1,0.25,0.25
2,0.50,0.50
3,0.75,0.75
4,1.00,1.00


## 표준화 (Standard Scailing)

평균이 0, 표준편차가 1이 되도록 변환

In [94]:
from sklearn.preprocessing import StandardScaler

In [95]:
standard_scaler = StandardScaler()

샘플 데이터 생성

In [102]:
x = np.arange(10)
# outiler 추가
x[9] = 1000
x

array([   0,    1,    2,    3,    4,    5,    6,    7,    8, 1000])

In [103]:
x.mean(), x.std()

(103.6, 298.8100399919654)

In [105]:
scaled = standard_scaler.fit_transform(x.reshape(-1,1))

In [106]:
x.mean(), x.std()

(103.6, 298.8100399919654)

In [107]:
scaled.mean(), scaled.std()

(4.4408920985006264e-17, 1.0)

In [108]:
round(scaled.mean(),2), scaled.std()

(0.0, 1.0)